Neo4j COVID-19 Clinical Trials Knowledge Graph Verification
This notebook verifies the restored Neo4j database from the paper:
"A PICO-based Knowledge Graph for Representing Clinical Evidence"

In [65]:
!pip install neo4j
from neo4j import GraphDatabase
import pandas as pd


In [67]:
# --- Define Connection Parameters ---
# The hostname 'adb-neo4j' resolves correctly inside the Jupyter container
URI = "bolt://localhost:7687"

# Authentication is an empty tuple because NEO4J_AUTH: none is set in docker-compose.yml
AUTH = ("neo4j", "neo4j")  # Default credentials for Neo4j when no auth is set

# --- Establish Connection and Query Executor ---
try:
    driver = GraphDatabase.driver(URI, auth=AUTH)
    driver.verify_connectivity()
    print("Connection to Neo4j successful! Ready to load data.")
except Exception as e:
    print(f"Connection failed. Please check if your 'adb-neo4j' container is running. Error: {e}")

def run_cypher_query(query):
    """Executes a Cypher query in a write transaction (suitable for DDL and data loading)."""
    with driver.session() as session:
        # Use execute_write for CREATE/MERGE operations
        result = session.execute_write(lambda tx: tx.run(query).data())
        return result


Connection to Neo4j successful! Ready to load data.


In [69]:
print("\n--- Step 1: Creating Constraints and Indexes ---")

CONSTRAINTS_AND_INDEXES = [
    "CREATE CONSTRAINT airports IF NOT EXISTS FOR (a:Airport) REQUIRE a.iata IS UNIQUE",
    "CREATE CONSTRAINT cities IF NOT EXISTS FOR (c:City) REQUIRE c.name IS UNIQUE",
    "CREATE CONSTRAINT regions IF NOT EXISTS FOR (r:Region) REQUIRE r.name IS UNIQUE",
    "CREATE CONSTRAINT countries IF NOT EXISTS FOR (c:Country) REQUIRE c.code IS UNIQUE",
    "CREATE CONSTRAINT continents IF NOT EXISTS FOR (c:Continent) REQUIRE c.code IS UNIQUE",
    "CREATE INDEX locations IF NOT EXISTS FOR (air:Airport) ON (air.location)"
]

try:
    for query in CONSTRAINTS_AND_INDEXES:
        run_cypher_query(query)
        
    print("Constraints and indexes created successfully.")

except Exception as e:
    print(f"Failed to create constraints/indexes: {e}")


print("\n--- Step 2: Loading Nodes and Geo-Hierarchy from airport-node-list.csv ---")

NODE_LOADING_QUERY = """
LOAD CSV WITH HEADERS FROM 'file:///airport-node-list.csv' AS row

MERGE (a:Airport {iata: row.iata})
MERGE (ci:City {name: row.city})
MERGE (r:Region {name: row.region})
MERGE (co:Country {code: row.country})
MERGE (con:Continent {name: row.continent})

MERGE (a)-[:IN_CITY]->(ci)
MERGE (a)-[:IN_COUNTRY]->(co)
MERGE (ci)-[:IN_COUNTRY]->(co)
MERGE (r)-[:IN_COUNTRY]->(co)
MERGE (a)-[:IN_REGION]->(r)
MERGE (ci)-[:IN_REGION]->(r)
MERGE (a)-[:ON_CONTINENT]->(con)
MERGE (ci)-[:ON_CONTINENT]->(con)
MERGE (co)-[:ON_CONTINENT]->(con)
MERGE (r)-[:ON_CONTINENT]->(con)

SET a.id = row.id,
    a.icao = row.icao,
    a.descr = row.descr,
    a.runways = toInteger(row.runways),
    a.longest = toInteger(row.longest),
    a.altitude = toInteger(row.altitude),
    a.latitude = toFloat(row.lat),
    a.longitude = toFloat(row.lon)
    // Removed any potential a.city property to keep city as a separate City label

RETURN count(a) AS airports_loaded
"""

results_nodes = run_cypher_query(NODE_LOADING_QUERY)
print(f"Airports and Geographies loaded. Total Airports: {results_nodes[0]['airports_loaded']}")

print("\n--- Step 3: Loading Routes from iroutes-edges.csv ---")

ROUTE_LOADING_QUERY = """
LOAD CSV WITH HEADERS FROM 'file:///iroutes-edges.csv' AS row

MATCH (source:Airport {iata: row.src})
MATCH (target:Airport {iata: row.dest})

MERGE (source)-[r:HAS_ROUTE]->(target)
ON CREATE SET r.distance = toInteger(row.dist)

RETURN count(r) AS routes_loaded
"""

results_routes = run_cypher_query(ROUTE_LOADING_QUERY)
print(f"Routes loaded. Total HAS_ROUTE relationships: {results_routes[0]['routes_loaded']}")



--- Step 1: Creating Constraints and Indexes ---
Constraints and indexes created successfully.

--- Step 2: Loading Nodes and Geo-Hierarchy from airport-node-list.csv ---
Airports and Geographies loaded. Total Airports: 3503

--- Step 3: Loading Routes from iroutes-edges.csv ---
Routes loaded. Total HAS_ROUTE relationships: 46389


In [59]:
def run_cypher_query(query):
    """Executes a read or write Cypher query."""
    with driver.session() as session:
        # Use a write transaction for data loading (CREATE/MERGE
        if query.strip().upper().startswith(("CREATE", "MERGE")):
             result = session.execute_write(lambda tx: tx.run(query).data())
        # Use a read transaction for fetching data (MATCH/RETURN)
        else:
             result = session.execute_read(lambda tx: tx.run(query).data())
        return result

In [70]:
with driver.session() as session:
    num_nodes = session.run("MATCH (n) RETURN count(n) AS c").single()["c"]
    num_rels = session.run("MATCH ()-[r]->() RETURN count(r) AS c").single()["c"]

print(f"Nombre de nœuds : {num_nodes}")
print(f"Nombre de relations : {num_rels}")

with driver.session() as session:
    labels = session.run("CALL db.labels()")
    print("Labels disponibles :")
    for record in labels:
        print("-", record["label"])



Nombre de nœuds : 8627
Nombre de relations : 73954
Labels disponibles :
- Airport
- City
- Region
- Country
- Continent


In [71]:
with driver.session() as session:
    rels = session.run("CALL db.relationshipTypes()")
    print("Types de relations :")
    for record in rels:
        print("-", record["relationshipType"])

Types de relations :
- IN_CITY
- IN_COUNTRY
- IN_REGION
- ON_CONTINENT
- HAS_ROUTE


In [72]:
label = "Airport" 
with driver.session() as session:
    result = session.run(f"MATCH (n:{label}) RETURN n LIMIT 10")
    df = pd.DataFrame([dict(record["n"]) for record in result])
    
df


,altitude,descr,longest,iata,latitude,icao,location,id,runways,longitude
0,1026,Hartsfield - Jackson Atlanta International Air...,12390,ATL,33.636700,KATL,"(-84.4281005859375, 33.6366996765137)",1,5,-84.428101
1,151,Anchorage Ted Stevens,12400,ANC,61.174400,PANC,"(-149.996002197266, 61.1744003295898)",2,3,-149.996002
2,542,Austin Bergstrom International Airport,12250,AUS,30.194500,KAUS,"(-97.6698989868164, 30.1944999694824)",3,2,-97.669899
3,599,Nashville International Airport,11030,BNA,36.124500,KBNA,"(-86.6781997680664, 36.1245002746582)",4,4,-86.678200
4,19,Boston Logan,10083,BOS,42.364300,KBOS,"(-71.00520325, 42.36429977)",5,6,-71.005203
5,143,Baltimore/Washington International Airport,10502,BWI,39.175400,KBWI,"(-76.66829681, 39.17539978)",6,3,-76.668297
6,14,Ronald Reagan Washington National Airport,7169,DCA,38.852100,KDCA,"(-77.0376968383789, 38.8521003723145)",7,3,-77.037697
7,607,Dallas/Fort Worth International Airport,13401,DFW,32.896801,KDFW,"(-97.0380020141602, 32.896800994873)",8,7,-97.038002
8,64,Fort Lauderdale/Hollywood International Airport,9000,FLL,26.072599,KFLL,"(-80.152702331543, 26.0725994110107)",9,2,-80.152702
9,313,Washington Dulles International Airport,11500,IAD,38.944500,KIAD,"(-77.45580292, 38.94449997)",10,4,-77.455803


In [50]:
# Explorer relation entre différents types de noeuds
with driver.session() as session:
    result = session.run("""
        MATCH (a:Airport)-[r:IN_CITY]->(b:City)
        RETURN a.iata AS source, b.name AS target, type(r) AS relation
        LIMIT 10
    """)
    df_rels = pd.DataFrame([record.data() for record in result])

df_rels 

,source,target,relation
0,ATL,Atlanta,IN_CITY
1,ANC,Anchorage,IN_CITY
2,AUS,Austin,IN_CITY
3,BNA,Nashville,IN_CITY
4,BOS,Boston,IN_CITY
5,BWI,Baltimore,IN_CITY
6,IAD,Washington D.C.,IN_CITY
7,DCA,Washington D.C.,IN_CITY
8,DAL,Dallas,IN_CITY
9,DFW,Dallas,IN_CITY


In [66]:
driver.close()
print("\nNeo4j connection closed.")


Neo4j connection closed.
